## 自動で抽出するアレ
- reference : [【https化に向けて】混在コンテンツのhttpを自動で抽出するやつを作った](http://www.procrasist.com/entry/get-http)

In [ ]:
#coding: utf-8
from bs4 import BeautifulSoup
import urllib
from urllib import request
import time
import csv
import os
from argparse import ArgumentParser

def extract_urls(root_url):
    """
    トップページを指定すると、ブログ内に存在するurlをすべて抜き出してくれる
    """
    is_articles = True
    page = 1
    urls = []
    titles = []
    while is_articles:
        try:
            html = request.urlopen("{}/archive?page={}".format(root_url, page))
        except urllib.error.HTTPError as e: 
            # HTTPレスポンスのステータスコードが404, 403, 401などの例外処理
            print(e.reason)
            break
        except urllib.error.URLError as e: 
            # アクセスしようとしたurlが無効なときの例外処理
            print(e.reason)
            break
        soup = BeautifulSoup(html, "html.parser")
        articles = soup.find_all("a",class_="entry-title-link")
        for article in articles:
            urls.append(article.get("href"))
            titles.append(article.text)
        if len(articles) == 0:
            is_articles = False
        page += 1
    return urls, titles

def get_http(i, urls, titles):
    """
    各記事内のhttpを保存
    - gif, jpg, jpeg, png
    """
    url = urls[i]
    title = titles[i]
    print(url,title)
    try:
        html = request.urlopen(url)
    except urllib.error.HTTPError as e: 
        print(e.reason)
    except urllib.error.URLError as e: 
        print(e.reason)
    soup = BeautifulSoup(html, "html.parser")
    entry = soup.select(".entry-content")[0]
    imgs = entry.find_all("img")
    scripts = entry.find_all("script")
    for script in scripts:
        url = script.get("src")
        if url == None:
            continue
        if "http://" in url:
                print("\tscript tag:",url)
    for img in imgs:
        url = img.get("src")
        if "http://" in url:
            print("\timg tag:",url)

def get_http_all(urls, titles):
    """
    各記事内のhttpを保存
    - gif, jpg, jpeg, png
    """
    for i, (url,title) in enumerate(zip(urls, titles)):
        count = 0
        try:
            html = request.urlopen(url)
        except urllib.error.HTTPError as e: 
            print(e.reason)
        except urllib.error.URLError as e: 
            print(e.reason)
        soup = BeautifulSoup(html, "html.parser")
        entry = soup.select(".entry-content")[0]
        imgs = entry.find_all("img")
        scripts = entry.find_all("script")
        for script in scripts:
            url = script.get("src")
            if url == None:
                continue
            if "http://" in url:
                if count == 0:
                    print(title,url)
                print("\tscript tag:",url)
                count += 1
        for img in imgs:
            url = img.get("src")
            if "http://" in url:
                if count == 0:
                    print(title,url)
                print("\timg tag:",url)
                count += 1

  ## この下にある、urlに自分のアドレスを入力して実行(必須)
 

In [ ]:
url = "http://www.procrasist.com"
urls, titles = extract_urls(url)
for i, (url,title) in enumerate(zip(urls, titles)):
    print(i, title, url)

## 全記事のhttpを取得したい方はこちらを実行

In [ ]:
get_http_all(urls, titles)

## ひと記事ずつ見たい方はこちらを実行
- 先ほどのリストに振られた番号を見ましょう
- i に見たい番号を代入しましょう

In [ ]:
i = 10
get_http(i, urls, titles)